# Simulating Constitutive Processes of semantic change within heterogeneous populations of speakers

In [1]:
# basic imports
import os
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# project code imports
from mod.one_hot_agent import *
from mod.plot import *
from mod.network import *

##### Hyper parameters

In [2]:
model_version = 'dy-allf'

In [3]:
no_agents = 2
no_connections = 1
add_vocab_in = 50
semantic_features = 3
starting_observations = 5
starting_uncertainty = .2
words_per_semantic_feature = 5
new_environment_prob = .25
enforce_word_feature_mapping = False

In [4]:
model_path = os.path.join('html',model_version)
if not os.path.exists(model_path):
    os.mkdir(model_path)

## Simple nteraction across repeated turns in a randomly generated environment

In [5]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [6]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [7]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [8]:
# and set the number of iterations
turns = 300

In [9]:
vocab_dif = []

In [10]:
for rd in tqdm(range(turns)):
    env = starting_env.sample()

    net.interaction(env, 3)
    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [11]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [12]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [13]:
fig.write_html(
    os.path.join(model_path,'Randomly-Generated-Environment.html')
)

## Random environment and introduction of new terms

In [14]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [15]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [16]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [17]:
# and set the number of iterations
turns = 300

In [18]:
vocab_dif = []

In [19]:
for rd in tqdm(range(turns)):
    env = starting_env.sample()

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [20]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [21]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.write_html(
    os.path.join(model_path,'Randomly-Generated-Environment-and-Introducing-Novel-Terms.html')
)
fig.show()

## Changing environment

In [22]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [23]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [24]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [25]:
# and set the number of iterations
turns = 300

In [26]:
vocab_dif = []

In [27]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [28]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [29]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [30]:
fig.write_html(
    os.path.join(model_path,'Suddenly-Changing-Environment.html')
)

## Changing environment and introduction of new terms

In [31]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [32]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [33]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [34]:
# and set the number of iterations
turns = 300

In [35]:
vocab_dif = []

In [36]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [37]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [38]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [39]:
fig.write_html(
    os.path.join(model_path,'Suddenly-Changing-Environment-and-Introducing-Novel-Terms.html')
)

## Stochastic environment

In [40]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [41]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [42]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [43]:
# and set the number of iterations
turns = 300

In [44]:
vocab_dif = []

In [45]:
for rd in tqdm(range(turns)):

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [46]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [47]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [48]:
fig.write_html(
    os.path.join(model_path,'Stochastically-Changing-Environment.html')
)

## Stochastic environment and introduction of new terms

In [49]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [50]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [51]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [52]:
# and set the number of iterations
turns = 30000

In [53]:
vocab_dif = []

In [54]:
for rd in tqdm(range(turns)):

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/30000 [00:00<?, ?it/s]

In [55]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [56]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [57]:
fig.write_html(
    os.path.join(model_path,'Stochastically-Changing-Environment-and-Introducing-Novel-Terms.html')
)

## Sudden and stochastic environment

In [58]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [59]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [60]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [61]:
# and set the number of iterations
turns = 300

In [62]:
vocab_dif = []

In [63]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [64]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [65]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [66]:
fig.write_html(
    os.path.join(model_path,'Suddenly-and-Stochastically-Changing-Environment.html')
)

## Sudden and stochastic environment, plus introduction of new terms

In [67]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [68]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [69]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [70]:
# and set the number of iterations
turns = 300

In [71]:
vocab_dif = []

In [72]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]

vocab_dif = pd.DataFrame(vocab_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [73]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [74]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [75]:
fig.write_html(
    os.path.join(model_path,'Suddenly-and-Stochastically-Changing-Environment-and-Introducing-Novel-Terms.html')
)